In [1]:
# CSCI 3832, Spring 2021, CU Boulder

from collections import Counter
from collections import defaultdict
import numpy as np
import re
from matplotlib import pyplot as plt
import sys

import collections
from math import log
import math




def create_ngrams(tokens, n): #creating ngrams function 
    ngrams = []
    for i in range(n-1,len(tokens)):
        ngram = tokens[i-n+1:i+1] 
        #Because it will stop right before i. Hence we put i+1 so that it goes through i.
        ngrams.append(tuple(ngram))
    return ngrams

def choose(prob, value):

    index = 0
    
    cum_prob = [0]* len(prob) #generating our cumulative "wheel" probability with prob that we collected
    cum_prob[0] = prob[0] #initialize the first as the same as prob array

    for k in range(1, len(prob)): #now start at 1 and go through and calculate the rest of the prob
        cum_prob[k] = cum_prob[k-1] + prob[k] #incrementally add with prob of i

    for k in range(len(cum_prob)): 
        if value < cum_prob[k]: #if our new generated value is within cum_prob[i]
            index = k  #save the index
            break
            
    return index



class LanguageModel:

    def __init__(self, ngram_order, is_laplace_smoothing):
        #your code
        
        self.vocab = {} #vocabulary of good words
        
        self.prob = {} #keeping track of a table of probabilities for bigram
        
        self.ngram_order = ngram_order #Value of N in N-gram (ex: 1 - 1-gram, 2 - 2-gram, 3 - 3-gram)
        
        self.is_laplace_smoothing = is_laplace_smoothing #boolean value if we use laplace smoothing or not
        
        self.ngram_counts = collections.Counter() #count of ngram count (for numerator probability calculations)
        
        self.ngram_1_counts = collections.Counter() # counting ngram-1 counts (for denominator probability calculations)
        
        self.test_ngram_counts = collections.Counter() #for test case
        
        self.test_ngram_1_counts = collections.Counter() #for test case
        
        #initialise the variables
        print("initialised \n")
            


    def train(self,filepath):
        UNK = "<UNK>"
        tokens = []
        ngram = []
        n_1gram = []
        
        
        with open(filepath) as file: #read the file
            lines = file.readlines()
        for l in lines: #read each word in each line
            token = l.split() 
            tokens.extend(token) #we will save it to our tokens array
        
        freq = collections.Counter() #counter to keep track of the freq/count of tokens - dictionary
        freq.update(tokens)
        #print("frequency: {} \n".format(freq)) #print out the frequency of each token to check it is parsing correctly
        
        bad_words = {k:v for k,v in freq.items() if v <= 1} #if count is <= 1, we will consider it a bad word
        self.vocab = {k:v for k,v in freq.items() if v > 1}
        
        for l in lines:    #if it is a bad word, we want to replace it with UNK
            token = l.split() #split our tokens/words
            for i in range (len(token)): #loop through each word
                if token[i] in bad_words: #if that word is a bad word
                    token[i] = UNK #then we assign that word to be unkown
                
            ngram.extend(create_ngrams(token, self.ngram_order)) #use createngrams function to find ngrams
            n_1gram.extend(create_ngrams(token, self.ngram_order - 1)) #use createngrams function to find ngrams-1
        
        self.ngram_counts.update(ngram) #update counts for the ngram
        self.ngram_1_counts.update(n_1gram) #update counts for ngram - 1  
    
            
        if(self.ngram_order >= 2):
            for n_tok in ngram: #we will now calculate probabilities
                n_count = self.ngram_counts[n_tok] #numerator
                n_1_count = self.ngram_1_counts[n_tok[:-1]] #denominator -don't consider last element  
                prob = n_count/n_1_count #use formula
                self.prob[n_tok] = prob
        else:
            for n_tok in ngram: #we will now calculate probabilities
                n_count = self.ngram_counts[n_tok] #numerator
                prob = (n_count)/ (len(self.vocab))
                self.prob[n_tok] = prob
            
            
        print('training done')
        print('num of unique {}-grams := {} \n'.format(self.ngram_order, self.ngram_counts))#todo: put the proper variables
 
    def score(self, sentence):
        # return the probability of the sentence
        
        UNK = "<UNK>"
        ngram = []
        token = sentence.split() #create our sentence
        
        for i in range (len(token)): #if our token is not in our vocab we will replace it with UNK token
            if token[i] not in self.vocab:
                token[i] = UNK
        
        #now we have a clean set of tokens:
        #create ngrams out of it
        ngram.extend(create_ngrams(token, self.ngram_order))
        
        p = 1 #not logspace. I added this so I could double check my answers with my handwritten answers
        
        for n_tok in ngram: #we will now calculate probabilities

            n_count = self.ngram_counts[n_tok] #numerator
            n_1_count = self.ngram_1_counts[n_tok[:-1]] #denominator -don't consider last element
            
            if(self.is_laplace_smoothing == False): #if we do not use laplace smoothing
                
                if(self.ngram_order >= 2):
                    prob = n_count/n_1_count #use formula
                    p = p * prob
                else:
                    prob = n_count/len(self.vocab)
                    p = p * prob
                
            
            else: #if we do use laplace smoothing
                prob = (n_count + 1)/ (n_1_count + len(self.vocab))
                p = p * prob
                
        return p
        

    def getPerplexity(self, filename):
        #return perplexity of the file
        #we gotta see the test set:
        
        test_token = []
        test_ngram = []
        test_n_1gram = []
        
        
        perplexity = 0
        UNK = "<UNK>"
        
        
        with open(filename) as file: #read the test file 
            lines = file.readlines()
        for l in lines: #read each word in each line of the test file
            token = l.split() 
            test_token.extend(token) #we will save it to our test tokens array
            
        freq = collections.Counter() #counter to keep track of the freq/count of tokens - dictionary
        freq.update(test_token)
        
        bad_words = {k:v for k,v in freq.items() if v <= 1} #if count is <= 1, we will consider it a bad word

        for l in lines:    #if it is a bad word, we want to replace it with UNK
            token = l.split() #split our tokens/words
            for i in range (len(test_token)): #loop through each word
                if test_token[i] in bad_words: #if that word is a bad word
                    test_token[i] = UNK #then we assign that word to be unkown
                
            test_ngram.extend(create_ngrams( token, self.ngram_order)) #use createngrams function to find ngrams
            test_n_1gram.extend(create_ngrams( token, self.ngram_order - 1)) #use createngrams function to find ngrams
        
        self.test_ngram_counts.update(test_ngram) #update counts for the ngram
        self.test_ngram_1_counts.update(test_n_1gram) #update counts for ngram - 1  
        
        
        n = len(test_token)
        
        for n_tok in test_ngram:
            
            n_count = self.test_ngram_counts[n_tok] #numerator
            n_1_count = self.test_ngram_1_counts[n_tok[:-1]] #denominator -don't consider last element
            
            if(self.ngram_order >= 2):
                prob = n_count/n_1_count
                probs = 1/prob
                perplexity = math.pow(probs, -(1/n))
            else:
                prob = n_count/len(self.vocab)
                probs = 1/prob
                perplexity = math.pow(probs, -(1/n))

        return perplexity

    def generate(self,num_sentences): #dont need laplace- generate words we see only
        
        s=[] 
        randomsentences = []
        

        for i in range(num_sentences):
            import random
            sentence = ['<s>',]
            prev  = '<s>'
            while(prev!='</s>'):
                
                collect = []
                probability = []
                
                if(self.ngram_order >= 2):

                    for m,n in self.prob.items():

                        if m[0] == prev:
                            collect.append(m)
                            probability.append(n)
                            
                    index = choose(probability, random.random())
                    bigram = collect[index]
                    sentence.append(bigram[1])
                    prev = bigram[1]
                    
                else:
                    for m,n in self.prob.items():
                        if m[0] != '<s>':
                            collect.append(m[0])
                            probability.append(n)
                    gram = random.choice(collect) #choose randomly from list without STRT and END
                    sentence.append(gram)
                    prev = gram

                if(prev == '</s>'):
                    break;
                    
            s = ' '.join(sentence)
            randomsentences.append(s)          
        
        return randomsentences

if __name__ == '__main__':
    

    # ADDED
    if len(sys.argv) != 3:
        print("Usage:", "python hw2_lm.py berp-training.txt hw2-test.txt ")
        sys.exit(1)

    trainingFilePath = sys.argv[1]
    testFilePath = sys.argv[2]



In [2]:
unigram = LanguageModel(1, True)
unigram.train("berp-training.txt")

f = open("hw2-unigram-generated.txt", "w")
g = open("hw2-unigram-out.txt", "w")
sent = unigram.generate(100)
for i in sent:
    f.write(i + "\n")
    g.write(str(unigram.score(i)) + "\n")
f.close()
g.close()

initialised 

training done
num of unique 1-grams := Counter({('<s>',): 6756, ('</s>',): 6756, ('i',): 2264, ('to',): 2153, ('like',): 1221, ('food',): 992, ('about',): 967, ('the',): 937, ('a',): 853, ('want',): 834, ('me',): 784, ('eat',): 662, ('would',): 652, ('restaurant',): 599, ('have',): 594, ('dollars',): 576, ("i'd",): 508, ('for',): 504, ('on',): 499, ('<UNK>',): 478, ('you',): 462, ('start',): 438, ('uh',): 431, ('more',): 418, ('over',): 414, ('of',): 406, ('tell',): 394, ('go',): 390, ('restaurants',): 386, ('can',): 381, ('dinner',): 362, ('is',): 359, ('any',): 340, ('some',): 323, ('in',): 312, ('than',): 311, ('be',): 304, ('lunch',): 299, ('it',): 276, ('ten',): 271, ('and',): 261, ('do',): 257, ('spend',): 249, ("i'm",): 246, ('information',): 242, ("let's",): 236, ('what',): 235, ('not',): 233, ('please',): 226, ('five',): 225, ('list',): 214, ('minutes',): 209, ('from',): 205, ('twenty',): 190, ('there',): 186, ('show',): 186, ('thai',): 184, ("don't",): 182, ('at

In [3]:
bigram = LanguageModel(2, False)
bigram.train("berp-training.txt")

a = open("hw2-bigram-generated.txt", "w")
b = open("hw2-bigram-out.txt",  "w")
sent = bigram.generate(100)
for i in sent:
    a.write(i + "\n")
    b.write(str(bigram.score(i)) + "\n")
a.close()
b.close()



initialised 

training done
num of unique 2-grams := Counter({('<s>', 'i'): 1653, ('like', 'to'): 953, ('i', 'want'): 743, ('food', '</s>'): 670, ('to', 'eat'): 610, ('i', 'would'): 607, ('would', 'like'): 585, ('want', 'to'): 546, ("i'd", 'like'): 495, ('dollars', '</s>'): 457, ('<s>', "i'd"): 434, ('start', 'over'): 403, ('tell', 'me'): 390, ('over', '</s>'): 367, ('to', 'go'): 294, ('restaurant', '</s>'): 286, ('<s>', 'tell'): 282, ('to', 'have'): 280, ('me', 'about'): 252, ('<s>', 'start'): 237, ('dinner', '</s>'): 224, ('<s>', "let's"): 196, ('do', 'you'): 193, ('to', 'spend'): 187, ('<s>', 'uh'): 184, ('<s>', "i'm"): 178, ('show', 'me'): 178, ('lunch', '</s>'): 175, ('<s>', 'what'): 172, ("let's", 'start'): 170, ('i', "don't"): 167, ('<s>', 'do'): 166, ('<s>', 'can'): 160, ('more', 'than'): 151, ('give', 'me'): 151, ('<s>', 'the'): 147, ('restaurants', '</s>'): 146, ('information', 'about'): 145, ('can', 'you'): 143, ('<UNK>', '</s>'): 137, ('less', 'than'): 136, ('can', 'i'): 13

In [4]:
print(unigram.getPerplexity("hw2-test.txt"))
print(bigram.getPerplexity("hw2-test.txt"))

0.9972271327209926
0.9986276771365581


In [8]:
trigram = LanguageModel(3, True)
trigram.train("berp-training - trigram.txt")
print(trigram.getPerplexity("hw2_test - trigram.txt"))

initialised 

training done
num of unique 3-grams := Counter({('<s>', '<s>', 'i'): 1653, ('food', '</s>', '</s>'): 670, ('<s>', 'i', 'want'): 651, ('i', 'would', 'like'): 582, ('<s>', 'i', 'would'): 511, ('would', 'like', 'to'): 474, ('i', 'want', 'to'): 474, ('dollars', '</s>', '</s>'): 457, ('<s>', '<s>', "i'd"): 434, ('<s>', "i'd", 'like'): 422, ("i'd", 'like', 'to'): 409, ('over', '</s>', '</s>'): 367, ('start', 'over', '</s>'): 366, ('like', 'to', 'eat'): 365, ('restaurant', '</s>', '</s>'): 286, ('<s>', '<s>', 'tell'): 282, ('<s>', 'tell', 'me'): 281, ('tell', 'me', 'about'): 248, ('<s>', '<s>', 'start'): 237, ('<s>', 'start', 'over'): 237, ('dinner', '</s>', '</s>'): 224, ('want', 'to', 'eat'): 199, ('like', 'to', 'have'): 197, ('<s>', '<s>', "let's"): 196, ('<s>', '<s>', 'uh'): 184, ('<s>', '<s>', "i'm"): 178, ('lunch', '</s>', '</s>'): 175, ('<s>', '<s>', 'what'): 172, ('<s>', '<s>', 'do'): 166, ('<s>', "let's", 'start'): 163, ('<s>', '<s>', 'can'): 160, ('like', 'to', 'go'): 